In [ ]:
import numpy as np
import keras
import tensorflow as tf
from keras import layers
from keras import models
from keras.datasets import california_housing

import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchinfo import summary
from torch.optim.lr_scheduler import StepLR

from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler

import sys
import math

import numpy as np
import math

import numpy as np
import keras
import tensorflow as tf
from keras import layers
from keras import models
from keras.datasets import mnist

import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchinfo import summary
from torch.optim.lr_scheduler import StepLR

import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder

import numpy as np
import pandas as pd
from scipy.special import logit
from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, accuracy_score
from sklearn.model_selection import LeaveOneOut, cross_val_predict, StratifiedKFold
from scipy.stats import norm

In [ ]:
class ReLUsubgrad(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input):
        ctx.save_for_backward(input)
        return input.clamp(min=0)

    @staticmethod
    def backward(ctx, grad_output):
        (input,) = ctx.saved_tensors
        grad_input = torch.zeros_like(input)
        
        grad_input[input > 0] = grad_output[input > 0]
        grad_input[input == 0] = grad_output[input == 0] * 0.5
        return grad_input

def relu_subgrad(x):
    return ReLUsubgrad.apply(x)

In [ ]:
class NegReLUsubgrad(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input):
        ctx.save_for_backward(input)
        return input.clamp(max=0)

    @staticmethod
    def backward(ctx, grad_output):
        (input,) = ctx.saved_tensors
        grad_input = torch.zeros_like(input)

        grad_input[input < 0] = grad_output[input < 0]
        grad_input[input == 0] = grad_output[input == 0] * 0.5
        return grad_input

def neg_relu_subgrad(x):
    return NegReLUsubgrad.apply(x)

In [ ]:
class CustomLinearLayer(nn.Module):
    def __init__(self, input_size, output_size, init="looks_linear"):
        super(CustomLinearLayer, self).__init__()
        self.linear = nn.Linear(input_size, output_size, bias=True)
        nn.init.zeros_(self.linear.bias)

        if init == "zero":
            nn.init.zeros_(self.linear.weight)
        elif init == "looks_linear":
            if input_size * 2 != output_size:
                raise ValueError("Output size must be twice that of input size")
                
            with torch.no_grad():
                weight = torch.zeros(input_size * 2, input_size)

                for i in range(self.linear.in_features):
                    weight[2 * i, i] = 1
                    weight[2 * i + 1, i] = -1

                self.linear.weight.copy_(weight)
                nn.init.zeros_(self.linear.bias)
                
            """ Example matrix: [
                [1, 0, 0],
                [-1, 0, 0],
                [0, 1, 0],
                [0, -1, 0],
                [0, 0, 1],
                [0, 0, -1]
            ] """

    def forward(self, x):
        return self.linear(x)

Showed some promising result in terms of generalization. Basic idea is to do gradient boosting on linear model first, then second model is second-order interaction between features, including a feature with itself, so n features require n * (n + 1) // 2 pairs, which are fed into the same amount of neurons where each neuron takes into two features, or a pair of features. This is, in constrast, to the traditional approach of feeding all features into each neuron, which enables neural network to do very high order feature interaction. Enabling higher-order feature interactions can be detrimental as higher-order feature interactions tend to be rarer and the network will be more likely to fit to noise

In [ ]:
class PairwiseNetwork(nn.Module):        
    def __init__(self, input_size, output_size, is_first_model):
        super(PairwiseNetwork, self).__init__()
        self.activation = nn.ReLU()
        
        self.input_size = input_size
        self.output_size = output_size
        
        self.is_first_model = is_first_model
                        
        if is_first_model:
            self.first_layer = CustomLinearLayer(input_size, output_size, init="zero")
        else:
            num_unique_pairs = input_size * (input_size + 1) // 2
            self.i_indices, self.j_indices = torch.triu_indices(input_size, input_size)

            self.weights = nn.Parameter(torch.zeros(1, output_size, num_unique_pairs, 2, 2))
            self.pairwise_biases = nn.Parameter(torch.zeros(output_size, num_unique_pairs, 2))
            self.outputs_biases = nn.Parameter(torch.zeros(1, output_size))
            
    def forward(self, x, prev_output=None):
        if self.is_first_model and prev_output is not None:
            raise ValueError("This is the first model and prev_output is passed in")
        if not self.is_first_model and prev_output is None:
            raise ValueError("This is not the first model and prev_output is not passed in")
        
        if prev_output is None:
            return self.first_layer(x)

        x_i = x[:, self.i_indices]
        x_j = x[:, self.j_indices]        
        feature_pairs = torch.stack([x_i, x_j], dim=2).unsqueeze(1).unsqueeze(-1)

        outputs = torch.matmul(self.weights, feature_pairs).squeeze(-1) + self.pairwise_biases
        outputs = relu_subgrad(outputs)
        outputs[:, :, :, 1] *= -1
        return outputs.sum(dim=(2, 3)) + self.outputs_biases

    def extract_features(self, x, prev_output=None):
        if self.is_first_model:
            raise ValueError("This is not intended for the first model")
        if not self.is_first_model and prev_output is None:
            raise ValueError("This is not the first model and prev_output is not passed in")
            
        return self.activation(self.first_layer(x))